In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", None)

df1 = pd.read_csv("labelling_BB.csv", delimiter=",")[["title", "abstract", "paper_id", "label"]]
df2 = pd.read_excel("labelling_PG.xlsx")[["title", "abstract", "paper_id", "label"]]
df3 = pd.read_excel("labelling_ET.xlsx")[["title", "abstract", "paper_id", "label"]]

# Clean DOI in df3
df3["paper_id"] = (
    df3["paper_id"]
    .astype(str)
    .str.extract(r"(10\.\d{4,9}/[-._;()/:A-Za-z0-9]+)", expand=False)
)

# Start with an empty 'label' column in df2 copy
df2_filled = df2.copy()
df2_filled["label"] = np.nan

# ---- 1) Match by paper_id ----
pid_map = df3.set_index("paper_id")["label"].to_dict()
df2_filled.loc[df2_filled["paper_id"].isin(pid_map), "label"] = df2_filled["paper_id"].map(pid_map)

# ---- 2) Match by title + abstract (lowercase) ----
# Prepare lookup dictionary
df3_pairs = df3.set_index(df3[["title", "abstract"]].apply(lambda x: (str(x[0]).lower(), str(x[1]).lower()), axis=1))["label"].to_dict()

mask_pairs = df2_filled["label"].isna()  # only fill missing labels
df2_filled.loc[mask_pairs, "label"] = (
    df2_filled.loc[mask_pairs, ["title", "abstract"]]
    .apply(lambda x: df3_pairs.get((str(x["title"]).lower(), str(x["abstract"]).lower())), axis=1)
)

# ---- 3) Match by title OR abstract individually (lowercase) ----
titles_map = df3.set_index(df3["title"].astype(str).str.lower())["label"].to_dict()
abstracts_map = df3.set_index(df3["abstract"].astype(str).str.lower())["label"].to_dict()

mask_remaining = df2_filled["label"].isna()

df2_filled.loc[mask_remaining, "label"] = (
    df2_filled.loc[mask_remaining].apply(
        lambda x: titles_map.get(str(x["title"]).lower())
        or abstracts_map.get(str(x["abstract"]).lower()),
        axis=1
    )
)

# ---- Result ----
df2_filled["selected_for_training"] = True
df2_filled.to_excel("labelling_ET_processed.xlsx", index=None)


C:\Users\k2371000\AppData\Local\Temp\ipykernel_15212\865699170.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df3_pairs = df3.set_index(df3[["title", "abstract"]].apply(lambda x: (str(x[0]).lower(), str(x[1]).lower()), axis=1))["label"].to_dict()
